In [ ]:
For this, you are going to copy the Vaihingen dataset in your own Google drive. Access the folder [here](https://drive.google.com/drive/folders/1Tr3q8kjPDzoamNFuHv7vTBsj5-acJC7Z?usp=sharing) and copy it to your drive.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls -F "/content/drive/MyDrive/Workshop_Semantic_Segmentation/DataVaihingen/top/"

In [ ]:
base_folder = '/content/drive/MyDrive/Workshop_Semantic_Segmentation'

In [ ]:
import os

# Define a consistent root directory for all data files (Fixes NameError: DATA_ROOT_DIR)
DATA_ROOT_DIR = os.path.join(base_folder, 'DataVaihingen')

print(f"DATA_ROOT_DIR set to: {DATA_ROOT_DIR}")

In [ ]:
import os

# --- Define standard Vaihingen image splits ---

# List of image area numbers to be used for training
TRAIN_AREAS = [1, 3, 5, 7, 11, 13, 15, 21, 23, 26, 30, 32, 34, 37]
# List of image area numbers to be used for validation (often called 'Test' in the data)
VAL_AREAS = [2, 4, 6, 8, 10, 12, 14, 16, 20, 22, 24, 27, 29, 31, 33, 35, 38]

def generate_file_list(data_root_dir, areas, output_filename):
    """Generates a text file listing all image names for the given areas."""

    # 1. Look for all .tif files in the 'top' directory
    img_dir = os.path.join(data_root_dir, 'top')
    if not os.path.exists(img_dir):
        print(f"Error: Image directory not found at {img_dir}. Check your DataVaihingen structure.")
        return

    all_files = os.listdir(img_dir)
    image_list = []

    # 2. Filter files to include only those matching the target areas
    for area in areas:
        # Standard file naming convention: top_mosaic_09cm_area[number].tif
        filename = f'top_mosaic_09cm_area{area}.tif'
        if filename in all_files:
            image_list.append(filename)

    # 3. Write the list to the specified output file
    output_path = os.path.join(data_root_dir, output_filename)
    with open(output_path, 'w') as f:
        f.write('\n'.join(image_list) + '\n')

    print(f"Generated {output_filename} with {len(image_list)} files at: {output_path}")


# --- Run Generation ---
print(f"Using DATA_ROOT_DIR: {DATA_ROOT_DIR}")

# Generate training list
generate_file_list(DATA_ROOT_DIR, TRAIN_AREAS, 'train_set.txt')

# Generate validation list
generate_file_list(DATA_ROOT_DIR, VAL_AREAS, 'val_set.txt')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Hypercolumns(nn.Module):
    def __init__(self, num_classes=6):
        super(Hypercolumns, self).__init__()

        # Encoder (VGG-like structure)
        # Block 1: Output size 256x256
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu1_1 = nn.ReLU(inplace=True)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.relu1_2 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

        # Block 2: Output size 128x128
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.relu2_1 = nn.ReLU(inplace=True)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu2_2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

        # Block 3: Output size 64x64
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.relu3_1 = nn.ReLU(inplace=True)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu3_2 = nn.ReLU(inplace=True)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.relu3_3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

        # Block 4: Output size 32x32
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.relu4_1 = nn.ReLU(inplace=True)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu4_2 = nn.ReLU(inplace=True)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu4_3 = nn.ReLU(inplace=True)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)

        # Block 5: Output size 16x16
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_1 = nn.ReLU(inplace=True)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_2 = nn.ReLU(inplace=True)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.relu5_3 = nn.ReLU(inplace=True)

        # 1x1 convolution layers for hypercolumns
        self.fc1 = nn.Conv2d(64, num_classes, kernel_size=1)
        self.fc2 = nn.Conv2d(128, num_classes, kernel_size=1)
        self.fc3 = nn.Conv2d(256, num_classes, kernel_size=1)
        self.fc4 = nn.Conv2d(512, num_classes, kernel_size=1)
        self.fc5 = nn.Conv2d(512, num_classes, kernel_size=1)

    def forward(self, x):

        # Block 1
        x = self.relu1_1(self.conv1_1(x))
        x = self.relu1_2(self.conv1_2(x))
        c1 = x # Feature map 1
        x, id1 = self.pool1(x)

        # Block 2
        x = self.relu2_1(self.conv2_1(x))
        x = self.relu2_2(self.conv2_2(x))
        c2 = x # Feature map 2
        x, id2 = self.pool2(x)

        # Block 3
        x = self.relu3_1(self.conv3_1(x))
        x = self.relu3_2(self.conv3_2(x))
        x = self.relu3_3(self.conv3_3(x))
        c3 = x # Feature map 3
        x, id3 = self.pool3(x)

        # Block 4
        x = self.relu4_1(self.conv4_1(x))
        x = self.relu4_2(self.conv4_2(x))
        x = self.relu4_3(self.conv4_3(x))
        c4 = x # Feature map 4
        x, id4 = self.pool4(x)

        # Block 5
        x = self.relu5_1(self.conv5_1(x))
        x = self.relu5_2(self.conv5_2(x))
        x = self.relu5_3(self.conv5_3(x))
        c5 = x # Feature map 5

        # Hypercolumn layers

        # c1 is 64x channels, 1/1 size
        hc1 = self.fc1(c1)

        # c2 is 128x channels, 1/2 size -> upscale to 1/1
        hc2 = F.interpolate(self.fc2(c2), size=c1.size()[2:], mode='bilinear', align_corners=False)

        # c3 is 256x channels, 1/4 size -> upscale to 1/1
        hc3 = F.interpolate(self.fc3(c3), size=c1.size()[2:], mode='bilinear', align_corners=False)

        # c4 is 512x channels, 1/8 size -> upscale to 1/1
        hc4 = F.interpolate(self.fc4(c4), size=c1.size()[2:], mode='bilinear', align_corners=False)

        # c5 is 512x channels, 1/16 size -> upscale to 1/1
        hc5 = F.interpolate(self.fc5(c5), size=c1.size()[2:], mode='bilinear', align_corners=False)

        # Combine hypercolumns
        hypercolumn_output = hc1 + hc2 + hc3 + hc4 + hc5

        return hypercolumn_output

In [ ]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import skimage.io as io
import os
import torch
from torchvision import transforms

class VaihingenDataset(Dataset):
    def __init__(self, img_folder, GT_folder, split='train', patch_size=256):

        # Define the full set of tiles
        all_train_tiles = [1, 3, 5, 7, 11, 13, 15, 17, 21, 23, 26, 28, 30, 32, 34, 37]
        all_val_tiles = [2, 4, 6, 8, 10, 12, 14, 16, 20, 22, 27, 29, 31, 33, 35, 38]

        if split == 'train':
            tiles_to_check = all_train_tiles
        elif split == 'val':
            tiles_to_check = all_val_tiles
        elif split == 'test':
            tiles_to_check = [2, 4, 6]
        else:
            raise ValueError("Split must be 'train', 'val', or 'test'")

        self.img_folder = img_folder
        self.GT_folder = GT_folder
        self.patch_size = patch_size
        self.split = split

        potential_files = []
        for i in tiles_to_check:
            fn = f'top_mosaic_09cm_area{i}.tif'
            if os.path.exists(os.path.join(img_folder, fn)) and os.path.exists(os.path.join(GT_folder, fn)):
                potential_files.append(fn)
            else:
                print(f"Warning: Tile {i} ({fn}) is missing and will be skipped.")

        if not potential_files:
            raise FileNotFoundError(f"No files found for split '{split}'. Check your data files.")

        loaded_images = [io.imread(os.path.join(img_folder, fn)) for fn in potential_files]
        loaded_GTs = [io.imread(os.path.join(GT_folder, fn)) for fn in potential_files]

        # --- CRITICAL FIX: Robust Shape and Patch Generation ---
        self.images = []
        self.GTs = []
        self.patch_locations = []
        new_tile_idx = 0

        for idx, img in enumerate(loaded_images):
            GT = loaded_GTs[idx]
            tile_name = potential_files[idx]

            # 1. Robust Image Shape Check
            try:
                H_img, W_img, C = img.shape
            except ValueError:
                print(f"Warning: Skipping tile {tile_name} (Image) due to malformed shape (H, W, C not found).")
                continue

            # 2. Robust GT Shape Check
            try:
                if len(GT.shape) == 2:
                    H_gt, W_gt = GT.shape
                else:
                    # Attempt to handle a 3D GT file (e.g., (H, W, 1))
                    H_gt, W_gt, _ = GT.shape
            except ValueError:
                print(f"Warning: Skipping tile {tile_name} (GT) due to malformed shape.")
                continue

            # 3. Compatibility Check (The source of the [0, 256] error)
            if H_img != H_gt or W_img != W_gt:
                print(f"Warning: Skipping tile {tile_name}. Image ({H_img}x{W_img}) and GT ({H_gt}x{W_gt}) shapes are incompatible.")
                continue

            H, W = H_img, W_img

            # 4. Size Check
            if H < self.patch_size or W < self.patch_size:
                print(f"Warning: Skipping tile {tile_name} due to size {H}x{W}. Too small for {self.patch_size} patch.")
                continue

            # If all checks pass, proceed to store and generate patches
            self.images.append(img)
            self.GTs.append(GT)

            # Generate patch coordinates for this valid tile
            rows = np.arange(0, H - self.patch_size + 1, self.patch_size)
            cols = np.arange(0, W - self.patch_size + 1, self.patch_size)

            for r in rows:
                for c in cols:
                    self.patch_locations.append((new_tile_idx, r, c))

            new_tile_idx += 1


        print(f"Dataset '{split}' initialized with {len(self.images)} valid tiles and {len(self.patch_locations)} total patches.")


    def __len__(self):
        return len(self.patch_locations)

    def __getitem__(self, idx):
        tile_idx, r, c = self.patch_locations[idx]

        img = self.images[tile_idx]
        GT = self.GTs[tile_idx]

        # Extract the patch (only the first 3 channels for RGB)
        patch_img = img[r:r + self.patch_size, c:c + self.patch_size, :3]
        patch_GT = GT[r:r + self.patch_size, c:c + self.patch_size]

        # Preprocessing
        patch_img = patch_img.astype(np.float32) / 255.0
        patch_img = np.transpose(patch_img, (2, 0, 1))

        img_tensor = torch.from_numpy(patch_img)
        GT_tensor = torch.from_numpy(patch_GT).long()

        return img_tensor, GT_tensor

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm

# =================================================================
# 1. METRICS FUNCTIONS
# =================================================================

def compute_confusion_matrix(pred, target, num_classes):
    """Computes the confusion matrix."""
    pred = pred.reshape(-1)
    target = target.reshape(-1)

    # Filter out the invalid class
    valid_mask = (target >= 0) & (target < num_classes)
    pred = pred[valid_mask]
    target = target[valid_mask]

    # Compute confusion matrix
    conf_matrix = np.zeros((num_classes, num_classes), dtype=np.int64)
    for t, p in zip(target.cpu().numpy(), pred.cpu().numpy()):
        conf_matrix[t, p] += 1

    return conf_matrix

def compute_metrics(conf_matrix):
    """Computes standard segmentation metrics from the confusion matrix."""
    metrics = {}

    TP = np.diag(conf_matrix)
    FP = conf_matrix.sum(axis=0) - TP
    FN = conf_matrix.sum(axis=1) - TP
    T = conf_matrix.sum()

    # Overall Accuracy (OA)
    metrics['Overall Accuracy'] = TP.sum() / T

    # Recall (Per-Class Accuracy)
    metrics['Recall'] = TP / (TP + FN + 1e-12)

    # Precision
    metrics['Precision'] = TP / (TP + FP + 1e-12)

    # F1 Score
    metrics['F1 Score'] = 2 * (metrics['Precision'] * metrics['Recall']) / (metrics['Precision'] + metrics['Recall'] + 1e-12)

    # Mean F1
    metrics['Mean F1'] = metrics['F1 Score'].mean()

    # Intersection over Union (IoU)
    metrics['IoU'] = TP / (TP + FP + FN + 1e-12)

    # Mean IoU
    metrics['Mean IoU'] = metrics['IoU'].mean()

    return metrics

def pretty_print_metrics(metrics):
    """Prints the metrics in a readable table format."""
    # NOTE: class_names are hardcoded for the Vaihingen dataset
    class_names = [
        "Impervious Surf.", "Building", "Low Vegetation",
        "Tree", "Car", "Clutter/Background"
    ]

    print("--------------------------------------------------")
    print(f"Overall Accuracy: {metrics['Overall Accuracy'] * 100:.2f}%")
    print(f"Mean F1 Score:    {metrics['Mean F1'] * 100:.2f}%")
    print(f"Mean IoU:         {metrics['Mean IoU'] * 100:.2f}%")
    print("--------------------------------------------------")
    print("Per-Class Metrics:")

    header = f"{'Class':<20} | {'IoU':<8} | {'F1 Score':<10}"
    print(header)
    print("-" * len(header))

    for i, name in enumerate(class_names):
        iou = metrics['IoU'][i] * 100
        f1 = metrics['F1 Score'][i] * 100
        print(f"{name:<20} | {iou:<8.2f} | {f1:<10.2f}")
    print("--------------------------------------------------")


# =================================================================
# 2. TRAINING AND VALIDATION FUNCTIONS
# =================================================================

def train_model(model, train_loader, criterion, optimizer, device):
    """Performs one epoch of training."""
    model.train()
    running_loss = 0.0

    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss


def val_model(model, val_loader, criterion, device, class_labels):
    """Performs validation and calculates loss and IoU."""
    model.eval()
    running_loss = 0.0
    conf_matrix = np.zeros((len(class_labels), len(class_labels)), dtype=np.int64)

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

            # Get predicted class (index of max probability)
            _, preds = torch.max(outputs, 1)

            # Update confusion matrix (Fixes NameError if not imported/defined)
            conf_matrix += compute_confusion_matrix(preds, labels, len(class_labels))

    epoch_loss = running_loss / len(val_loader.dataset)

    # Calculate metrics
    metrics = compute_metrics(conf_matrix)
    mean_iou = metrics['Mean IoU']

    return epoch_loss, mean_iou


# =================================================================
# 3. INFERENCE/PREDICTION FUNCTION
# =================================================================

def produce_results(image_name):
    """
    Inference function to load an image, predict, and display results.
    NOTE: Requires DATA_ROOT_DIR and base_folder to be correctly defined.
    """
    import os
    import skimage.io as io
    import matplotlib.pyplot as plt

    # --- Corrected Path Logic (Uses globally defined DATA_ROOT_DIR) ---
    image_path = os.path.join(DATA_ROOT_DIR, 'top', image_name)
    GT_path    = os.path.join(DATA_ROOT_DIR, '1CGT', image_name)
    # --- End Corrected Path Logic ---

    # Load data
    image = io.imread(image_path)
    image_rgb = image[:, :, :3]
    GT = io.imread(GT_path).astype(int)

    # Preprocessing for the model
    img_tensor = image_rgb.astype(np.float32) / 255.0
    img_tensor = np.transpose(img_tensor, (2, 0, 1))
    img_tensor = torch.from_numpy(img_tensor).unsqueeze(0).to(device)

    # Inference
    network.eval()
    with torch.no_grad():
        output = network(img_tensor)

    # Post-processing
    output = output.squeeze().cpu()

    _, predicted_mask = torch.max(output, 0)

    predicted_mask = predicted_mask.numpy()

    # Plotting
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(image_rgb)
    plt.title(f'Original Image: {image_name}')

    plt.subplot(1, 3, 2)
    plt.imshow(GT)
    plt.title('Ground Truth')

    plt.subplot(1, 3, 3)
    plt.imshow(predicted_mask)
    plt.title('Predicted Mask')

    plt.show()

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
# NOTE: VaihingenDataset and Hypercolumns are assumed to be defined in previous cells (Code 7 and Code 6)

# =========================================================
# 1. Configuration and Paths
# =========================================================

# Paths confirmed in previous steps. Ensure these paths are correct in your Google Drive.
DATA_ROOT_DIR = "/content/drive/MyDrive/Workshop_Semantic_Segmentation/DataVaihingen"
base_folder = "/content/drive/MyDrive/Workshop_Semantic_Segmentation"

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define Hyperparameters
class_labels = [0, 1, 2, 3, 4, 5]
patch_size = 256
batch_size = 4
number_epochs = 25 # Changed to 25 based on your training loop code

# Define the folder paths within the root directory
IMG_FOLDER = os.path.join(DATA_ROOT_DIR, 'top')
GT_FOLDER = os.path.join(DATA_ROOT_DIR, '1CGT')


# =========================================================
# 2. Initialize Data Loaders (CRITICAL FIX APPLIED)
# =========================================================

# Create Datasets (Uses the fixed VaihingenDataset from Code 7)
train_data = VaihingenDataset(IMG_FOLDER, GT_FOLDER, split='train', patch_size=patch_size)
val_data = VaihingenDataset(IMG_FOLDER, GT_FOLDER, split='val', patch_size=patch_size)

# Create DataLoaders
# *** CRITICAL FIX: Setting num_workers=0 to prevent the RuntimeError ***
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=0)


# =========================================================
# 3. Initialize Model, Loss, and Optimizer
# =========================================================

# Initialize Model (Hypercolumns is assumed to be defined in Code 6)
network = Hypercolumns(num_classes=len(class_labels))
network.to(device)
print("Network initialized and moved to device.")

# Initialize Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=0.001)

print("Criterion and Optimizer initialized.")

In [ ]:
import torch
import os
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
import numpy as np

# NOTE: This code assumes that the network, loaders, criterion, optimizer,
# and the correct training/validation functions (train_model, val_model)
# have been initialized and defined in the preceding cells (Code 8 and Code 9).

# =========================================================================
# SETUP
# =========================================================================

# Variables (Assumed from Code 9)
number_epochs = 25
base_folder = "/content/drive/MyDrive/Workshop_Semantic_Segmentation"
# device, network, train_loader, val_loader, criterion, optimizer, class_labels are ready

# Setup for Model Saving
save_directory = os.path.join(base_folder, 'WeightsVaihingen')
os.makedirs(save_directory, exist_ok=True)
best_model_path = os.path.join(save_directory, 'best_hypercolumns_model.pth')

# Metric Initialization
train_losses = []
val_losses = []
val_ious = []
best_iou = -1.0

# =========================================================================
# BASELINE TRAINING LOOP (25 EPOCHS)
# =========================================================================

print(f"Starting baseline training for {number_epochs} epochs...")
for epoch in range(number_epochs):
    print(f"\n--- Epoch {epoch+1}/{number_epochs} ---")

    # Training step (uses the correct train_model function from Code 8)
    train_loss = train_model(network, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    print(f"Training Loss: {train_loss:.4f}")

    # Validation step (uses the correct val_model function from Code 8)
    val_loss, val_iou = val_model(network, val_loader, criterion, device, class_labels)
    val_losses.append(val_loss)
    val_ious.append(val_iou)

    print(f"Validation Loss: {val_loss:.4f}, Mean IoU: {val_iou:.4f}")

    # Save the best model based on Mean IoU
    if val_iou > best_iou:
        best_iou = val_iou
        # Save model state dict to the designated Drive location
        torch.save(network.state_dict(), best_model_path)
        print(f"Model saved to {best_model_path} with new best IoU: {best_iou:.4f}")

print("\nTraining complete.")

In [ ]:
import torch
from tqdm.notebook import tqdm # tqdm was imported in Cell 6

# Assuming the variables (network, train_loader, val_loader, criterion, optimizer, device) are all defined

train_losses = []
val_losses = []
val_ious = []
best_iou = -1.0
best_model_path = 'best_hypercolumns_model.pth' # The path where your best model weights will be saved

print("Starting training...")
for epoch in range(number_epochs):
    print(f"\n--- Epoch {epoch+1}/{number_epochs} ---")

    # Training step (uses train_model from Cell 6)
    train_loss = train_model(network, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    print(f"Training Loss: {train_loss:.4f}")

    # Validation step (uses val_model from Cell 6)
    val_loss, val_iou = val_model(network, val_loader, criterion, device, class_labels)
    val_losses.append(val_loss)
    val_ious.append(val_iou)
    print(f"Validation Loss: {val_loss:.4f}, Mean IoU: {val_iou:.4f}")

    # Save the best model based on Mean IoU
    if val_iou > best_iou:
        best_iou = val_iou
        # Save model state dict
        torch.save(network.state_dict(), best_model_path)
        print(f"Model saved to {best_model_path} with new best IoU: {best_iou:.4f}")

print("\nTraining complete.")

In [ ]:
## corrected code

import numpy as np
import torch
from tqdm.notebook import tqdm

# NOTE: This function relies on 'compute_confusion_matrix' and 'compute_metrics'
# which must be defined in your metric calculation cells.

def val_model(network, val_loader, criterion, device, class_labels):
    # 1. Setup
    network.eval() # Set the model to evaluation mode
    total_val_loss = 0.0
    num_classes = len(class_labels)

    # Initialize a fresh confusion matrix for this epoch's validation run
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=np.int64)

    with torch.no_grad(): # Disable gradient calculations for validation
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 2. Forward Pass and Loss
            # Using the criterion passed into the function (the unweighted one for baseline)
            outputs = network(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

            # 3. Get Predictions
            _, preds = torch.max(outputs, 1)

            # 4. Accumulate Confusion Matrix
            # This function must be defined in your metrics cells
            confusion_matrix += compute_confusion_matrix(preds, labels, num_classes)

    # 5. Calculate Final Metrics (Mean IoU)
    avg_val_loss = total_val_loss / len(val_loader)

    # This function must be defined in your metrics cells and return a dictionary
    metrics = compute_metrics(confusion_matrix)
    # The compute_metrics function likely returns IoU as a percentage (38.60),
    # so we divide by 100.0 to return a float for correct comparison with best_iou = -1.0
    mean_iou = metrics['Mean IoU'] / 100.0

    # Return the two required values
    return avg_val_loss, mean_iou

In [ ]:
import matplotlib.pyplot as plt

# --- Plot Loss ---
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Loss History Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss (Cross Entropy)')
plt.legend()
plt.grid(True)

# --- Plot IoU ---
plt.subplot(1, 2, 2)
plt.plot(val_ious, label='Validation Mean IoU', color='red')
plt.title('Validation Mean IoU Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Mean IoU')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import torch

# 1. Load the Best Model Weights
# NOTE: Ensure best_model_path points to the correct location (e.g., on Drive if you used base_folder)
print(f"Loading weights from: {best_model_path}")
network.load_state_dict(torch.load(best_model_path))
network.to(device)
network.eval()
print("Best model weights loaded successfully.")

# 2. Run Inference on a Sample Validation Image
# We will use an image from the validation set (e.g., area10)
sample_image_name = 'top_mosaic_09cm_area10.tif'

# The produce_results function (defined in Cell 8) handles loading, prediction, and plotting.
print(f"\nGenerating results for {sample_image_name}...")
produce_results(sample_image_name)

In [ ]:
import numpy as np
import torch
from tqdm.notebook import tqdm

# Ensure the best model is loaded and in evaluation mode
network.load_state_dict(torch.load(best_model_path))
network.to(device)
network.eval()
print("Starting full validation set evaluation...")

# Initialize a confusion matrix for all validation data
total_confusion_matrix = np.zeros((len(class_labels), len(class_labels)), dtype=np.int64)

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = network(inputs)

        # Get predicted class
        _, preds = torch.max(outputs, 1)

        # Accumulate confusion matrix
        total_confusion_matrix += compute_confusion_matrix(preds, labels, len(class_labels))

# Calculate final metrics from the aggregated confusion matrix
final_metrics = compute_metrics(total_confusion_matrix)

print("\n--- Final Metrics on Validation Set ---")
# The pretty_print_metrics function (defined in Cell 8) displays the final table
pretty_print_metrics(final_metrics)

print("\nEvaluation complete.")

In [ ]:
import torch
import numpy as np

# Class labels defined in Cell 9: [0, 1, 2, 3, 4, 5]
num_classes = len(class_labels)

# Initialize counts for each class
class_counts = np.zeros(num_classes, dtype=np.int64)

print("Calculating class frequencies from training data...")

# Iterate through the entire training dataset to count pixels for each class
for _, labels in tqdm(train_data, desc="Counting Pixels"):
    # Convert labels from tensor to numpy array
    labels_np = labels.numpy()

    # Count pixels for each class (0 to 5)
    for i in range(num_classes):
        class_counts[i] += np.sum(labels_np == i)

print("\nPixel counts per class:")
print(class_counts)

# Calculate weights: Inverse frequency scaling (median frequency balancing is also common)
# Weight = Total Pixels / Class Count
total_pixels = class_counts.sum()
raw_weights = total_pixels / (class_counts + 1e-12) # Add small epsilon to prevent division by zero

# Normalize weights so they sum up to the number of classes (optional, but good practice)
weights = raw_weights / raw_weights.sum() * num_classes

# Convert weights to a PyTorch tensor and move to the device
class_weights = torch.from_numpy(weights).float().to(device)

print("\nCalculated Class Weights (to be used in loss function):")
print(class_weights)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Redefine the criterion using the calculated weights
criterion_weighted = nn.CrossEntropyLoss(weight=class_weights)
print("Weighted CrossEntropyLoss initialized.")

# Redefine the optimizer, resetting learning rate and state
optimizer_weighted = optim.Adam(network.parameters(), lr=0.001)
print("Optimizer reset.")

In [ ]:


import torch
import os
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim

# NOTE: Requires base_folder, criterion_weighted, optimizer_weighted, criterion (unweighted),
# train_loader, val_loader, and class_labels to be defined in previous cells.

# --- Setup for Model Saving (New Path for Weighted Model) ---
# NOTE: Ensure 'base_folder' is defined for persistent saving
save_directory = os.path.join(base_folder, 'WeightsVaihingen')
os.makedirs(save_directory, exist_ok=True)
best_model_path_weighted = os.path.join(save_directory, 'best_hypercolumns_weighted_model.pth')

# --- Metric Initialization ---
train_losses_w = []
val_losses_w = []
val_ious_w = []
best_iou_w = -1.0

print(f"Starting Weighted Training for {number_epochs} epochs...")
for epoch in range(number_epochs): # This loop now runs 25 times
    print(f"\n--- Epoch {epoch+1}/{number_epochs} (Weighted) ---")

    # Training step - using the weighted criterion and optimizer
    train_loss = train_model(network, train_loader, criterion_weighted, optimizer_weighted, device)
    train_losses_w.append(train_loss)
    print(f"Training Loss: {train_loss:.4f}")

    # Validation step - always use the unweighted criterion for fair loss comparison
    val_loss, val_iou = val_model(network, val_loader, criterion, device, class_labels)
    val_losses_w.append(val_loss)
    val_ious_w.append(val_iou)
    print(f"Validation Loss: {val_loss:.4f}, Mean IoU: {val_iou:.4f}")

    # Save the best model based on Mean IoU
    if val_iou > best_iou_w:
        best_iou_w = val_iou
        # Save model state dict to the designated Drive location
        torch.save(network.state_dict(), best_model_path_weighted)
        print(f"Model saved to {best_model_path_weighted} with new best IoU: {best_iou_w:.4f}")

print("\nWeighted Training complete.")

In [ ]:
import numpy as np
import torch
from tqdm.notebook import tqdm

# The path to the best model from the weighted training run (defined in the last step)
# NOTE: Ensure best_model_path_weighted is defined and points to the correct weights!

# 1. Load the Best Weighted Model Weights
print(f"Loading weighted model weights from: {best_model_path_weighted}")
network.load_state_dict(torch.load(best_model_path_weighted))
network.to(device)
network.eval()

# 2. Run Full Evaluation
print("\nStarting full validation set evaluation of the WEIGHTED MODEL...")

# Initialize a fresh confusion matrix
total_confusion_matrix_weighted = np.zeros((len(class_labels), len(class_labels)), dtype=np.int64)

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating Weighted"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = network(inputs)
        _, preds = torch.max(outputs, 1)

        # Accumulate confusion matrix
        total_confusion_matrix_weighted += compute_confusion_matrix(preds, labels, len(class_labels))

# 3. Calculate and Print Final Metrics
final_metrics_weighted = compute_metrics(total_confusion_matrix_weighted)

print("\n--- Final Metrics on Validation Set (WEIGHTED LOSS) ---")
# Use the pretty print function from the original notebook's Cell 31 (now Cell 31/32 equivalent)
pretty_print_metrics(final_metrics_weighted)

print("\nEvaluation of weighted model complete.")

In [ ]:
import torch
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

# We will use the best weighted model
network.load_state_dict(torch.load(best_model_path_weighted))
network.to(device)
network.eval()

def tiled_prediction_and_display(network, image_name, patch_size=256, overlap=32, device=device):
    """
    Performs inference using an overlapping tile strategy and displays results.
    """

    # --- 1. Setup ---
    image_path = os.path.join(DATA_ROOT_DIR, 'top', image_name)
    GT_path    = os.path.join(DATA_ROOT_DIR, '1CGT', image_name)

    # Load data
    image = io.imread(image_path)
    image_rgb = image[:, :, :3]
    GT = io.imread(GT_path).astype(int)

    H, W, C = image_rgb.shape

    # Initialize the output prediction map
    predicted_mask = np.zeros((H, W), dtype=np.uint8)

    # Define stride for tiling
    stride = patch_size - overlap

    # Calculate starting points for rows and columns
    r_starts = np.arange(0, H, stride)
    c_starts = np.arange(0, W, stride)

    # Adjust starts to ensure the last patch fully covers the image edge
    if r_starts[-1] + patch_size < H:
        r_starts = np.append(r_starts, H - patch_size)
    else:
        r_starts[-1] = H - patch_size

    if c_starts[-1] + patch_size < W:
        c_starts = np.append(c_starts, W - patch_size)
    else:
        c_starts[-1] = W - patch_size

    r_starts = np.unique(r_starts)
    c_starts = np.unique(c_starts)

    total_tiles = len(r_starts) * len(c_starts)

    print(f"Total tiles to process: {total_tiles} ({len(r_starts)} rows x {len(c_starts)} cols)")


    with torch.no_grad():
        for r_start in tqdm(r_starts, desc="Tiling Rows"):
            for c_start in c_starts:

                # Extract patch
                patch_img = image_rgb[r_start:r_start + patch_size, c_start:c_start + patch_size, :3]

                # Preprocessing
                img_tensor = patch_img.astype(np.float32) / 255.0
                img_tensor = np.transpose(img_tensor, (2, 0, 1))
                img_tensor = torch.from_numpy(img_tensor).unsqueeze(0).to(device)

                # Inference
                output = network(img_tensor)

                # Post-processing
                output = output.squeeze().cpu()
                _, pred_patch = torch.max(output, 0)
                pred_patch = pred_patch.numpy().astype(np.uint8)

                # --- 3. Stitching (Simple Overwrite) ---
                r_end = r_start + patch_size
                c_end = c_start + patch_size

                predicted_mask[r_start:r_end, c_start:c_end] = pred_patch

    # --- 4. Plotting ---
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(image_rgb)
    plt.title(f'Original Image: {image_name}')

    plt.subplot(1, 3, 2)
    plt.imshow(GT)
    plt.title('Ground Truth')

    plt.subplot(1, 3, 3)
    plt.imshow(predicted_mask)
    plt.title('Tiled Predicted Mask (Weighted Model)')

    plt.show()

# Run the Tiled Inference on a sample image (e.g., area10 from validation set )
sample_image_name = 'top_mosaic_09cm_area10.tif'
tiled_prediction_and_display(network, sample_image_name, patch_size=256, overlap=32, device=device)

U NET

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. Basic Convolutional Block
class DoubleConv(nn.Module):
    """(Convolution => BN => ReLU) * 2"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

# 2. Downsampling Block
class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

# 3. Corrected Upsampling Block (FIXED: Handles channels for x1 and x2 separately)
class Up(nn.Module):
    """Upscaling then double conv with correct channel calculation"""
    def __init__(self, in_channels_x1, in_channels_x2, out_channels):
        super().__init__()

        # 1. The upsampling layer operates only on x1 channels
        self.up = nn.ConvTranspose2d(in_channels_x1, in_channels_x1, kernel_size=2, stride=2)

        # 2. The subsequent double convolution takes the concatenated tensor
        # Total channels = x2 (skip connection) + x1_upsampled
        self.conv = DoubleConv(in_channels_x2 + in_channels_x1, out_channels)

    def forward(self, x1, x2):
        # Upsample x1
        x1 = self.up(x1)

        # Concatenate x2 (skip connection) and x1 (upsampled features)
        x = torch.cat([x2, x1], dim=1)

        return self.conv(x)

# 4. Final U-Net Model (FIXED: Instantiates Up blocks with correct channel arguments)
class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=6):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        # Encoder (Contracting Path)
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)

        # Decoder (Expanding Path) - MODIFIED CHANNEL CALLS
        # Up(in_channels_x1, in_channels_x2, out_channels)
        self.up1 = Up(512, 512, 256)
        self.up2 = Up(256, 256, 128)
        self.up3 = Up(128, 128, 64)
        self.up4 = Up(64, 64, 64)

        # Output Layer
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        # Encoder
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)

        # Decoder + Skip Connections
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)

        # Final Classification
        logits = self.outc(x)
        return logits

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
# NOTE: VaihingenDataset and helper functions (Code 7 & 8) must be defined and run prior to this.

# =========================================================
# 1. Configuration and Paths
# =========================================================

# Ensure your paths are correct in your Google Drive.
DATA_ROOT_DIR = "/content/drive/MyDrive/Workshop_Semantic_Segmentation/DataVaihingen"
base_folder = "/content/drive/MyDrive/Workshop_Semantic_Segmentation"

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define Hyperparameters
class_labels = [0, 1, 2, 3, 4, 5]
patch_size = 256
batch_size = 4
number_epochs = 25

# Define the folder paths within the root directory
IMG_FOLDER = os.path.join(DATA_ROOT_DIR, 'top')
GT_FOLDER = os.path.join(DATA_ROOT_DIR, '1CGT')


# =========================================================
# 2. Initialize Data Loaders
# =========================================================

# Create Datasets (VaihingenDataset from Code 7)
train_data = VaihingenDataset(IMG_FOLDER, GT_FOLDER, split='train', patch_size=patch_size)
val_data = VaihingenDataset(IMG_FOLDER, GT_FOLDER, split='val', patch_size=patch_size)

# Create DataLoaders (with num_workers=0 fix)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=0)


# =========================================================
# 3. Initialize Model, Loss, and Optimizer
# =========================================================

# Initialize Model: UNet is instantiated here
network = UNet(n_channels=3, n_classes=len(class_labels))
network.to(device)
print("U-Net Network initialized and moved to device.")

# Initialize Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=0.001)

print("Criterion and Optimizer initialized.")

In [ ]:
import torch
import os
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
import numpy as np

# =========================================================================
# SETUP
# =========================================================================

# Variables (Assumed from Code 9)
number_epochs = 25
base_folder = "/content/drive/MyDrive/Workshop_Semantic_Segmentation"

# Setup for Model Saving - SAVES TO GOOGLE DRIVE
save_directory = os.path.join(base_folder, 'WeightsVaihingen_UNet')
os.makedirs(save_directory, exist_ok=True)
best_model_path = os.path.join(save_directory, 'best_UNet_model.pth')

# Metric Initialization
train_losses = []
val_losses = []
val_ious = []
best_iou = -1.0

# =========================================================================
# BASELINE TRAINING LOOP
# =========================================================================

print(f"Starting U-Net training for {number_epochs} epochs...")
for epoch in range(number_epochs):
    print(f"\n--- Epoch {epoch+1}/{number_epochs} ---")

    # Training step
    train_loss = train_model(network, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    print(f"Training Loss: {train_loss:.4f}")

    # Validation step
    val_loss, val_iou = val_model(network, val_loader, criterion, device, class_labels)
    val_losses.append(val_loss)
    val_ious.append(val_iou)

    print(f"Validation Loss: {val_loss:.4f}, Mean IoU: {val_iou:.4f}")

    # Save the best model based on Mean IoU
    if val_iou > best_iou:
        best_iou = val_iou
        # Save model state dict to the designated Drive location
        torch.save(network.state_dict(), best_model_path)
        print(f"Model saved to {best_model_path} with new best IoU: {best_iou:.4f}")

print("\nTraining complete.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import os

# =========================================================================
# 1. Load the Best Model Weights
# =========================================================================

# Ensure the best_model_path points to the file saved by the training loop
best_model_path = os.path.join(base_folder, 'WeightsVaihingen_UNet', 'best_UNet_model.pth')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Re-instantiate the UNet model (which was defined in Code 1)
try:
    # NOTE: The UNet class must be defined in your notebook scope
    network.load_state_dict(torch.load(best_model_path, map_location=device))
    network.eval()
    print(f"Successfully loaded best UNet model from: {best_model_path}")
except FileNotFoundError:
    print(f"Error: Model file not found at {best_model_path}. Did training finish?")
except NameError:
    print("Error: The 'UNet' class definition must be executed before loading the model.")

# =========================================================================
# 2. Plot Training History
# =========================================================================

def plot_metrics(train_losses, val_losses, val_ious):
    """Plots the loss and IoU curves over epochs."""
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(15, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'r-o', label='Training Loss')
    # Check if val_losses is populated before plotting
    if val_losses and any(v is not None for v in val_losses):
        plt.plot(epochs, val_losses, 'b-o', label='Validation Loss')

    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot IoU
    plt.subplot(1, 2, 2)
    # Ensure val_ious is a list of numbers
    if val_ious and all(isinstance(i, (int, float)) for i in val_ious):
        plt.plot(epochs, val_ious, 'g-o', label='Mean IoU')
        plt.title('Mean Intersection over Union (IoU)')
        plt.xlabel('Epochs')
        plt.ylabel('IoU')
        plt.legend()
        plt.grid(True)
    else:
        plt.title('Mean IoU (Data Missing or Invalid)')
        print("\nWarning: IoU metrics were not plotted correctly. Check your val_model return values.")

    plt.show()

print("\nPlotting training history...")
plot_metrics(train_losses, val_losses, val_ious)

# =========================================================================
# 3. Visualization Placeholder (Requires your test image data)
# =========================================================================
print("\n--- Model Comparison Summary ---")
print(f"Best U-Net Mean IoU achieved: {best_iou:.4f}")
# You should manually compare this to the best_iou from your Hypercolumns run!

# NOTE: This section requires you to load a test image and its ground truth
# to display a side-by-side comparison.

"""
# --- Visualization Placeholder ---

# 1. Load a test image (e.g., test_input) and its ground truth (e.g., test_gt)
#    from your test set. (This data loading code needs to be added by you.)
# test_input, test_gt = load_single_test_image()

# 2. Preprocess and run inference
# with torch.no_grad():
#     test_input = test_input.to(device).unsqueeze(0) # Add batch dimension
#     output = network(test_input)
#     _, prediction = torch.max(output, 1)
#     prediction = prediction.cpu().squeeze(0).numpy()

# 3. Display results
# visualize_prediction(test_input.cpu().squeeze(0), test_gt, prediction)
"""
print("Proceed to add your final visualization code to compare input, GT, and prediction.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import os

# =========================================================================
# 1. Load the Best Model Weights
# =========================================================================

# Ensure the best_model_path points to the file saved by the training loop
best_model_path = os.path.join(base_folder, 'WeightsVaihingen_UNet', 'best_UNet_model.pth')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Re-instantiate the UNet model (which was defined in Code 1)
try:
    # NOTE: The UNet class must be defined in your notebook scope
    network.load_state_dict(torch.load(best_model_path, map_location=device))
    network.eval()
    print(f"Successfully loaded best UNet model from: {best_model_path}")
except FileNotFoundError:
    print(f"Error: Model file not found at {best_model_path}. Did training finish?")
except NameError:
    print("Error: The 'UNet' class definition must be executed before loading the model.")

# =========================================================================
# 2. Plot Training History
# =========================================================================

def plot_metrics(train_losses, val_losses, val_ious):
    """Plots the loss and IoU curves over epochs."""
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(15, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'r-o', label='Training Loss')
    # Check if val_losses is populated before plotting
    if val_losses and any(v is not None for v in val_losses):
        plt.plot(epochs, val_losses, 'b-o', label='Validation Loss')

    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot IoU
    plt.subplot(1, 2, 2)
    # Ensure val_ious is a list of numbers
    if val_ious and all(isinstance(i, (int, float)) for i in val_ious):
        plt.plot(epochs, val_ious, 'g-o', label='Mean IoU')
        plt.title('Mean Intersection over Union (IoU)')
        plt.xlabel('Epochs')
        plt.ylabel('IoU')
        plt.legend()
        plt.grid(True)
    else:
        plt.title('Mean IoU (Data Missing or Invalid)')
        print("\nWarning: IoU metrics were not plotted correctly. Check your val_model return values.")

    plt.show()

print("\nPlotting training history...")
plot_metrics(train_losses, val_losses, val_ious)

# =========================================================================
# 3. Model Comparison Summary
# =========================================================================
print("\n--- Model Comparison Summary ---")
print(f"Best U-Net Mean IoU achieved: {best_iou:.4f}")
print("Compare this Mean IoU to your Hypercolumns Mean IoU to determine which model performed better!")
print("Next, add your custom code for visual inspection of a test image.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import os

# -------------------------------------------------------------------------
# HELPER FUNCTION FOR VISUALIZATION (Color Mapping - MUST be run here)
# -------------------------------------------------------------------------

def visualize_prediction(image, gt, prediction, iou):
    """
    Plots the input image, ground truth, and prediction mask.
    """
    # RGB color mapping for the 6 classes
    COLOR_MAP = np.array([
        [255, 255, 255], # 0: Impervious Surfaces (White)
        [0, 0, 255],     # 1: Building (Blue)
        [0, 255, 255],   # 2: Low Vegetation (Cyan)
        [0, 255, 0],     # 3: Tree (Green)
        [255, 255, 0],   # 4: Car (Yellow)
        [255, 0, 0]      # 5: Clutter/Background (Red)
    ], dtype=np.uint8)

    def map_index_to_color(mask):
        color_mask = np.zeros((*mask.shape, 3), dtype=np.uint8)
        for i in range(len(COLOR_MAP)):
            color_mask[mask == i] = COLOR_MAP[i]
        return color_mask

    # 1. Prepare data for plotting
    image_np = image.permute(1, 2, 0).cpu().numpy()
    gt_colored = map_index_to_color(gt)
    pred_colored = map_index_to_color(prediction)

    # 2. Plotting
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    axes[0].imshow(image_np)
    axes[0].set_title('Input Image (RGB)')
    axes[0].axis('off')

    axes[1].imshow(gt_colored)
    axes[1].set_title('Ground Truth')
    axes[1].axis('off')

    axes[2].imshow(pred_colored)
    axes[2].set_title(f'U-Net Prediction (Mean IoU: {iou:.4f})')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

# -------------------------------------------------------------------------
# INFERENCE EXECUTION
# -------------------------------------------------------------------------

print("\nRunning visual inspection on a validation patch for U-Net...")

# 1. Grab one patch from the validation dataset
try:
    val_iterator = iter(val_loader)
    inputs, labels = next(val_iterator)
except NameError:
    print("Error: val_loader is not defined. Ensure Code 9 was run.")
    raise

# Select the first image in the batch for visualization
input_patch = inputs[0]
gt_mask = labels[0].cpu().numpy()

# 2. Preprocess and Run Inference
network.to(device)
network.eval()
with torch.no_grad():
    input_tensor = input_patch.to(device).unsqueeze(0)
    output = network(input_tensor)
    _, prediction_mask = torch.max(output, 1)
    prediction_mask = prediction_mask.cpu().squeeze(0).numpy()

# 3. Visualize
# Assuming best_iou holds the U-Net's best IoU value from Code 11
visualize_prediction(input_patch, gt_mask, prediction_mask, best_iou)

print("\nU-Net visual comparison generated.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import os

# NOTE: The Hypercolumns class MUST be defined in your notebook scope
# (from the initial phase of the project).

# -------------------------------------------------------------------------
# 1. Load the Best Hypercolumns Model Weights
# -------------------------------------------------------------------------

# Path for the Hypercolumns weights (Assuming the default folder name from the first run)
hypercolumns_best_path = os.path.join(base_folder, 'WeightsVaihingen', 'best_model.pth')
print(f"Attempting to load Hypercolumns model from: {hypercolumns_best_path}")

# Re-instantiate the Hypercolumns model
try:
    # IMPORTANT: You must replace 'Hypercolumns' with the actual class name if it was different.
    hypercolumns_network = Hypercolumns(num_classes=len(class_labels))
    hypercolumns_network.load_state_dict(torch.load(hypercolumns_best_path, map_location=device))
    hypercolumns_network.to(device)
    hypercolumns_network.eval()
    print("Successfully loaded and set Hypercolumns model to evaluation mode.")

except NameError:
    print("\nError: The 'Hypercolumns' class is not defined. Please define it and run again.")
    raise
except FileNotFoundError:
    print(f"\nError: Hypercolumns model file not found at {hypercolumns_best_path}. Check save path.")
    raise

# Placeholder for Hypercolumns IoU (REPLACE WITH YOUR ACTUAL VALUE)
# You MUST manually find and replace '0.5000' with the best IoU score achieved by your Hypercolumns model!
hypercolumns_best_iou = 0.5000
print(f"Hypercolumns Best IoU placeholder: {hypercolumns_best_iou:.4f}")

# -------------------------------------------------------------------------
# INFERENCE EXECUTION
# -------------------------------------------------------------------------

print("\nRunning visual inspection for the Hypercolumns model on the same validation patch...")

# 1. Run Inference (using the same input_patch tensor loaded in the previous block)
with torch.no_grad():
    input_tensor = input_patch.to(device).unsqueeze(0) # Input is the same
    output = hypercolumns_network(input_tensor)
    _, prediction_mask = torch.max(output, 1)
    prediction_mask = prediction_mask.cpu().squeeze(0).numpy()

# 2. Visualize (using the same visualization function from the previous block)
# The function is defined in your previous cell and is now ready to use.
visualize_prediction(input_patch, gt_mask, prediction_mask, hypercolumns_best_iou)

print("\nVisual comparison for Hypercolumns complete.")

# -------------------------------------------------------------------------
# 3. Final Summary
# -------------------------------------------------------------------------
print("\n--- Project Conclusion ---")
print("You have successfully trained, analyzed, and visualized both the Hypercolumns and U-Net models.")
print("Final comparison:")
print(f"1. U-Net Best Mean IoU: {best_iou:.4f}")
print(f"2. Hypercolumns Best Mean IoU: {hypercolumns_best_iou:.4f}")
print("\nCompare the two prediction images visually and compare the IoU scores to draw your conclusion.")